In [16]:
import xarray as xr
import numpy as np
import sys
sys.path.append( '/groups/sylvia/JAS-MCS-rain/' )
from thermodynamic_functions import *

In [2]:
eps = 0.01802/0.02897 # ratio of MW_water to MW_air
basedir = '/xdisk/sylvia/DYAMOND-UM/'
pressh_file = basedir + 'phalf_3hr_HadGEM3-GA71_N2560_20160801-20160908.nc_tropics.nc'
pressf_file = basedir + 'pfull_3hr_HadGEM3-GA71_N2560_20160801-20160908.nc_tropics.nc'

### Start by regridding the 3D files from model levels to pressure levels

In [11]:
# calculate the mean pressure at a given model level over time, lat, lon for the two datasets above
pressh = xr.open_dataset(pressh_file)['air_pressure']
pressf = xr.open_dataset(pressf_file)['air_pressure']

mean_pressh = np.empty([55,])
mean_pressf = np.empty([55,])

for i in np.arange(55):
    mean_pressh[i] = pressh[:,i].mean(dim={'time','longitude','latitude'}).values
    mean_pressf[i] = pressf[:,i].mean(dim={'time','longitude','latitude'}).values

# write the mean pressure at a given model level over time, lat, lon to a text file
# these files output pressures from high/surface values to low/TOA values
np.savetxt( '/groups/sylvia/JAS-MCS-rain/DYAMOND/mean_phalf_3hr_HadGEM3-GA71_N2560_20160801-20160908.txt',
          mean_pressh )
np.savetxt( '/groups/sylvia/JAS-MCS-rain/DYAMOND/mean_pfull_3hr_HadGEM3-GA71_N2560_20160801-20160908.txt',
          mean_pressf )

### Reading in regridded 3D files

In [12]:
# 3D pressure level HadGEM data from DYAMOND
# vertical velocity [m s-1], temperature [K], 
# suffixed _PLfull or _PLhalf based on which pressure file they were regridded from
w_pfile = basedir + 'wa_3hr_HadGEM3-GA71_N2560_20160801-20160908_PLfull.nc'
t_pfile = basedir + 'ta_3hr_HadGEM3-GA71_N2560_20160801-20160908_PLfull.nc'
qv_pfile = basedir + 'hus_3hr_HadGEM3-GA71_N2560_20160801-20160908_PLfull.nc'
#cli_file = basedir + 'cli_3hr_HadGEM3-GA71_N2560_20160801-20160908_PLfull.nc'
#clw_file = basedir + 'clw_3hr_HadGEM3-GA71_N2560_20160801-20160908_PLfull.nc'

# 2D HadGEM data from DYAMOND
#precip_file = basedir + ''
olr_file = basedir + 'rlut_1hr_HadGEM3-GA71_N2560_20160801-20160908.nc_tropics.nc'

In [ ]:
# dimensions for the variables going into the scaling
time = xr.open_dataset(w_pfile)['time']        # every 3 hours for 39 days (312 values)
lats = xr.open_dataset(w_pfile)['latitude']    # -5 to 5 (214 values)
lons = xr.open_dataset(w_pfile)['longitude']   # -180 to 180 (5120 values)

In [10]:
# variables going into the scaling -- vertical velocity, temperature, specific humidity
w_HadGEM = xr.open_dataset(w_pfile)['upward_air_velocity']  # (312 times, 55 levels, 214 lat, 5120 lon)
t_HadGEM = xr.open_dataset(t_pfile)['air_temperature']
qv_HadGEM = xr.open_dataset(qv_pfile)[]

<xarray.DataArray 'air_temperature' (time: 312, model_level_number: 55, latitude: 214, longitude: 5120)>
[18801868800 values with dtype=float32]
Coordinates:
  * time                (time) datetime64[ns] 2016-08-01T03:00:00 ... 2016-09-09
  * longitude           (longitude) float32 -180.0 -179.9 -179.8 ... 179.9 180.0
  * latitude            (latitude) float32 -4.992 -4.945 -4.898 ... 4.945 4.992
  * model_level_number  (model_level_number) int32 1 2 3 4 5 ... 51 52 53 54 55
Attributes:
    standard_name:  air_temperature
    long_name:      TEMPERATURE ON THETA LEVELS
    units:          K

### Calculate the efficiency over the whole long channel

In [ ]:
# Start by evaluating the saturation vapor mixing ratio from qv and T
qvsat_HadGEM = eps * satVapP_liq(t_HadGEM) / ( plev_ERAI - satVapP_liq(t_HadGEM) )

# Then calculate the gradient versus pressure in that saturation vapor mixing ratio
